# I. Load Scores via ESPN API

In [1]:
from espn_functions import *
#Define week and year
week = 3
year = 2023
standings_df, matchup_df, HP_Owner, HP_Player, HP_Score, HT_Owner, HT_Score = run_espn_weekly(week, year)

Data not loaded correctly (Attempt 1 of 5), retrying in 1 second...
Loaded data from ESPN for week 3 of the 2023 season
Completed processing weekly scores and standings
Saved week 3 matchup data


# II. Create GPT Summary Email

In [5]:
summary = generate_summary(week, matchup_df, standings_df)
print(summary)

Generated the summary
Hey there, Fantasy Football Fanatics! Here's a rundown of the Pro League Pummelings, Harrowing Hits, and Epic Fails from Week 3. Grab your popcorn, folks!

Kicking things off with a bang (more like a thermonuclear explosion), we have "King Drew" (Andrew Kim) who took the crown (quite literally) marching to the top with a high score of 137.08. The Bills' DST carrying the day with 31 points. Quite a draft pick, Andrew, your crystal ball deserves a standing ovation. 

"One sketch, two sketch, red sketch, Not Sketch" (Kai Shen) busted out a 128.8 this week, starring Raheem Mostert who surged with 41.7 points. As unpredictable as Dr. Seuss rhythm, but hey it's working!

"Can you Digg it, Sucka?" (Tom Zhang) fought valiantly but couldn’t dig their way out against "NACHOS TEAM" (Justin Chen), both shy of a hundred, scraping 96.3 and 97.06 respectively. What happened, guys? Did you let your cats pick your lineups this week?

Meanwhile "Good Things Bad People" (Dan Jiang) 

# III. Send G-Mail

In [6]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']
recipients = list(config_data['emails']['espn_email_dict'].values())

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

#Format the tables
scoring_tabulate = tabulate(matchup_df, headers='keys', tablefmt='html', showindex=False)
standings_tabulate = tabulate(standings_df, headers='keys', tablefmt='html', showindex=False)

# Define message
subject = f"Weekly Fagudas FF Recap: Week {week}"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p><b>Highest Scoring Team</b>: {HT_Owner}: {HT_Score} points</p>
    <p><b>Highest Scoring Player</b>: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings_tabulate}
  </body>
</html>
"""


#recipients = ['lithal06@yahoo.com']
sender = user

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection vu4-20020a05620a560400b00774309d3e89sm1903092qkn.7 - gsmtp')